<a href="https://colab.research.google.com/github/alilotfi90/A-Natural-Language-Processing-Journey/blob/main/steam-review-sequential-model-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding



nltk.download('punkt')
nltk.download('stopwords')

drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from sklearn.model_selection import train_test_split
import nltk

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:
!unzip "/content/drive/MyDrive/steam_data_set.zip" -d "/content/"

Archive:  /content/drive/MyDrive/steam_data_set.zip
  inflating: /content/test_gr/test.csv  
  inflating: /content/train_gr/game_overview.csv  
  inflating: /content/train_gr/train.csv  


In [4]:
# Load data into a Pandas DataFrame
data_path = "/content/train_gr/train.csv"
df = pd.read_csv(data_path)

# Explore first few rows of the DataFrame
print(df.head())

   review_id                        title    year  \
0          1  Spooky's Jump Scare Mansion  2016.0   
1          2  Spooky's Jump Scare Mansion  2016.0   
2          3  Spooky's Jump Scare Mansion  2016.0   
3          4  Spooky's Jump Scare Mansion  2015.0   
4          5  Spooky's Jump Scare Mansion  2015.0   

                                         user_review  user_suggestion  
0  I'm scared and hearing creepy voices.  So I'll...                1  
1  Best game, more better than Sam Pepper's YouTu...                1  
2  A littly iffy on the controls, but once you kn...                1  
3  Great game, fun and colorful and all that.A si...                1  
4  Not many games have the cute tag right next to...                1  


In [5]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and case normalization
    tokens = [word for word in tokens if word.isalpha()]  # Keep words only
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(tokens)

df['cleaned_review'] = df['user_review'].apply(preprocess_text)

df.head()

,review_id,title,year,user_review,user_suggestion,cleaned_review
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1,scared hearing creepy voices pause moment writ...
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1,best game better sam pepper youtube account ne...
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1,littly iffy controls know play easy master mad...
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1,great game fun colorful side note though getti...
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1,many games cute tag right next horror tag firs...


In [6]:


# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned_review'])
sequences = tokenizer.texts_to_sequences(df['cleaned_review'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

# Build LSTM Model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=100),
    LSTM(32),
    Dense(1, activation='sigmoid')
])




In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['user_suggestion'].values, test_size=0.3, random_state=42)

# List of epochs
epoch_list = [5, 10, 20]

# Model Training with different epochs
for epoch in epoch_list:
    print(f'\nTraining model for {epoch} epochs...\n')

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch, batch_size=32)

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print(f'\nTest accuracy after {epoch} epochs:', test_acc, '\n')



Training model for 5 epochs...

Epoch 1/5
383/383 [==============================] - 60s 137ms/step - loss: 0.6784 - accuracy: 0.5763 - val_loss: 0.6259 - val_accuracy: 0.6626
Epoch 2/5
383/383 [==============================] - 20s 53ms/step - loss: 0.5768 - accuracy: 0.7225 - val_loss: 0.5800 - val_accuracy: 0.7293
Epoch 3/5
383/383 [==============================] - 13s 33ms/step - loss: 0.5288 - accuracy: 0.7537 - val_loss: 0.5547 - val_accuracy: 0.7478
Epoch 4/5
383/383 [==============================] - 13s 34ms/step - loss: 0.5746 - accuracy: 0.7078 - val_loss: 0.5686 - val_accuracy: 0.7434
Epoch 5/5
383/383 [==============================] - 8s 21ms/step - loss: 0.5051 - accuracy: 0.7870 - val_loss: 0.5440 - val_accuracy: 0.7641
165/165 - 1s - loss: 0.5440 - accuracy: 0.7641 - 602ms/epoch - 4ms/step

Test accuracy after 5 epochs: 0.76414555311203 


Training model for 10 epochs...

Epoch 1/10
383/383 [==============================] - 5s 14ms/step - loss: 0.4571 - accuracy: 0.